In [0]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)

In [0]:
n_train = mnist.train.num_examples  # 55,000
n_validation = mnist.validation.num_examples  # 5000
n_test = mnist.test.num_examples  # 10,000

We have divided the data set into 3 sections


In [0]:
n_input = 784  # input layer (28x28 pixels)
n_hidden1 = 512  # 1st hidden layer
n_hidden2 = 256  # 2nd hidden layer
n_hidden3 = 128  # 3rd hidden layer
n_output = 10  # output layer (0-9 digits)

5 layer architecture with 3 hidden layers with number of units as shown above

Converted a test sample of image of 28*28 pixel into 1D vector


In [0]:
learning_rate = 1e-4
n_iterations = 1000
batch_size = 128
dropout = 0.5

We have set the hyper parameters which will remain constant throughout the execution

In [0]:
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_output])
keep_prob = tf.placeholder(tf.float32)

tensors in tensorflow are data structures which are passed to the graph and get updated during the learning process

We used 3 tensors as placeholders
These placeholder can be initialized with just their size and no data and later the data will be fed to them

If data within the variable keeps on changing then always prefer placeholders 

In [0]:
weights = {
    'w1': tf.Variable(tf.truncated_normal([n_input, n_hidden1], stddev=0.1)),
    'w2': tf.Variable(tf.truncated_normal([n_hidden1, n_hidden2], stddev=0.1)),
    'w3': tf.Variable(tf.truncated_normal([n_hidden2, n_hidden3], stddev=0.1)),
    'out': tf.Variable(tf.truncated_normal([n_hidden3, n_output], stddev=0.1)),
}

In [0]:
biases = {
    'b1': tf.Variable(tf.constant(0.1, shape=[n_hidden1])),
    'b2': tf.Variable(tf.constant(0.1, shape=[n_hidden2])),
    'b3': tf.Variable(tf.constant(0.1, shape=[n_hidden3])),
    'out': tf.Variable(tf.constant(0.1, shape=[n_output]))
}

Eventhough the weights and bias values will also get updated during the learning process but it is advisable to initialize them with some random values rather than keeping empty.

In [0]:
layer_1 = tf.add(tf.matmul(X, weights['w1']), biases['b1'])
layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
layer_3 = tf.add(tf.matmul(layer_2, weights['w3']), biases['b3'])
layer_drop = tf.nn.dropout(layer_3, keep_prob)
output_layer = tf.matmul(layer_3, weights['out']) + biases['out']

A popular choice of loss function in TensorFlow programs is cross-entropy, also known as log-loss, similar to cost function .

In [0]:
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(
        labels=Y, logits=output_layer
        ))

Cross entropy will return us the cost of the model

In [0]:
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

Tensorflow will be using Adam optimizer as gradient descent optimization function to minimize cost function.

In [0]:
correct_pred = tf.equal(tf.argmax(output_layer, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

For measuring correctness of the model we will be maintaining vector of boolean values whether the model predicted it correct or not , for which we will be comparing oputput layer is equal to out Y 

Note here we have a vector of size 10 with one hot criteria so to find the value we will select the max using argmax and return corresponding value for it like 
[0,0,0,0,1,0,0,0,0,0] = 5  

Type cast these boolean values into float and find the mean which will act as accuracy for our model.

In [0]:
init = tf.global_variables_initializer()

Initialized the hyper parameters before starting the training.

Now create a session for our training model  and run it for setting all the initial parameters

In [0]:
sess = tf.Session()
sess.run(init)

Now we will be iterating the model for n_iterations (=1000) and print the loss and accuracy for each multiple of 100 and we will notice the loss reduces and accuracy increases if the model is functioning as per our expectations or else we need to consider changing some parameters and train again.

In [15]:
# train on mini batches
for i in range(n_iterations):
    batch_x, batch_y = mnist.train.next_batch(batch_size)
    sess.run(train_step, feed_dict={
        X: batch_x, Y: batch_y, keep_prob: dropout
        })

    # print loss and accuracy (per minibatch)
    if i % 100 == 0:
        minibatch_loss, minibatch_accuracy = sess.run(
            [cross_entropy, accuracy],
            feed_dict={X: batch_x, Y: batch_y, keep_prob: 1.0}
            )
        print(
            "Iteration",
            str(i),
            "\t| Loss =",
            str(minibatch_loss),
            "\t| Accuracy =",
            str(minibatch_accuracy)
            )

Iteration 0 	| Loss = 3.3921785 	| Accuracy = 0.09375
Iteration 100 	| Loss = 0.42636597 	| Accuracy = 0.875
Iteration 200 	| Loss = 0.2800888 	| Accuracy = 0.953125
Iteration 300 	| Loss = 0.2610808 	| Accuracy = 0.9296875
Iteration 400 	| Loss = 0.3240548 	| Accuracy = 0.875
Iteration 500 	| Loss = 0.3049673 	| Accuracy = 0.890625
Iteration 600 	| Loss = 0.4677275 	| Accuracy = 0.859375
Iteration 700 	| Loss = 0.23484246 	| Accuracy = 0.9375
Iteration 800 	| Loss = 0.4439272 	| Accuracy = 0.8671875
Iteration 900 	| Loss = 0.2773733 	| Accuracy = 0.9140625


Model is able to predict with acuracy >90%

We have trained the model on our training set , now we will try it on test set to check overfitting or whether it is able to predict with high accuracy.

In [16]:
test_accuracy = sess.run(accuracy, feed_dict={X: mnist.test.images, Y: mnist.test.labels, keep_prob: 1.0})
print("\nAccuracy on test set:", test_accuracy)


Accuracy on test set: 0.9166


Completed first Neural Network training model with an accuracy of 92% !!!

To further improve the performance and accuracy we can adjust the values of hyper parameters and test the effect on accuracy.

In [0]:
import numpy as np
from PIL import Image

Trying it for an image of 28X28 pixel image given by us.

In [25]:
!curl -o two.png https://raw.githubusercontent.com/do-community/tensorflow-digit-recognition/master/test_img.png
img = np.invert(Image.open("two.png").convert('L')).ravel()

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   393  100   393    0     0   4792      0 --:--:-- --:--:-- --:--:--  4792


In [26]:
prediction = sess.run(tf.argmax(output_layer, 1), feed_dict={X: [img]})
print ("Prediction for test image:", np.squeeze(prediction))

Prediction for test image: 2


Hurray it predicted two 

In [31]:
from IPython.display import Image
Image('two.png')
Image("two.png", width=100, height=100)